In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.stats import spearmanr

CONSULTATION_MAP = {1: "Consultation", 2: "Diagnosis", 3: "Mortality"}

def evaluate_top_cases_by_type(test_year, test_month):
    """Evaluate prediction accuracy for each consultation type separately."""
    global df, model, X_train_columns

    # Filter training/test data
    train_df = df[(df['Year'] < test_year) | ((df['Year'] == test_year) & (df['Month'] < test_month))]
    test_df = df[(df['Year'] == test_year) & (df['Month'] == test_month)]
    if test_df.empty:
        print(f"No data available for {test_year}-{test_month:02d}")
        return None

    print(f"\n=== Evaluating predictions for {test_year}-{test_month:02d} ===")
    results = []

    for consult_type in sorted(test_df['Consultation_Type'].unique()):
        type_name = CONSULTATION_MAP.get(consult_type, f"Type {consult_type}")
        print(f"\n🩺 Consultation Type {consult_type}: {type_name}")

        train_type_df = train_df[train_df['Consultation_Type'] == consult_type]
        test_type_df = test_df[test_df['Consultation_Type'] == consult_type]
        if len(test_type_df) < 10:
            print(f"  ⚠ Skipping — not enough data for {type_name}")
            continue

        X_train = train_type_df.drop(columns=['Total'])
        y_train = train_type_df['Total']
        X_test  = test_type_df.drop(columns=['Total'])
        y_test  = test_type_df['Total']

        # Train a temporary model for that type
        rf = RandomForestRegressor(
            n_estimators=100,
            random_state=42,
            n_jobs=-1,
            min_samples_leaf=5
        )
        rf.fit(X_train, y_train)

        # Predict on test set
        y_pred = rf.predict(X_test)
        y_pred[y_pred < 0] = 0
        test_type_df = test_type_df.copy()
        test_type_df['Predicted_Total'] = y_pred

        # Regression metrics
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Compute actual & predicted top cases
        actual_top = (
            test_type_df.groupby('Case')['Total'].sum()
            .sort_values(ascending=False)
            .head(10)
            .index.tolist()
        )

        predicted_top = (
            test_type_df.groupby('Case')['Predicted_Total'].sum()
            .sort_values(ascending=False)
            .head(10)
            .index.tolist()
        )

        overlap = len(set(actual_top) & set(predicted_top))
        top10_accuracy = overlap / 10 * 100

        # Rank correlation
        merged = pd.merge(
            test_type_df.groupby('Case')['Total'].sum(),
            test_type_df.groupby('Case')['Predicted_Total'].sum(),
            left_index=True, right_index=True
        )
        spearman_corr, _ = spearmanr(merged['Total'], merged['Predicted_Total'])

        # Print results
        print(f"  MAE: {mae:.3f}")
        print(f"  R²: {r2:.3f}")
        print(f"  Top-10 Overlap: {overlap}/10 ({top10_accuracy:.1f}%)")
        print(f"  Spearman Rank Corr: {spearman_corr:.3f}")
        print(f"  Actual Top: {actual_top}")
        print(f"  Predicted Top: {predicted_top}")

        results.append({
            "consult_type": consult_type,
            "consult_type_name": type_name,
            "mae": mae,
            "r2": r2,
            "top10_overlap": overlap,
            "top10_accuracy_percent": top10_accuracy,
            "spearman_corr": spearman_corr
        })

    return results
import pandas as pd

df = pd.read_csv("master_dataset_cleaned.csv")
results = evaluate_top_cases_by_type(2024, 12)


NameError: name 'df' is not defined